# Agenda

1. More with reading from and writing to files
2. Cleaning data with `nan` and interpolating
3. Analysis with data frames
    - Cutting and categorizing
    - Sorting
    - Grouping
    - Concatenating data frames together
    - Join data frames
    